In [1]:
import time
import threading
import heapq
import random
from uuid import uuid4

class User:
    def __init__(self, name, location, is_priority=False):
        self.name = name
        self.location = location  
        self.is_priority = is_priority

class Driver:
    def __init__(self, name, location):
        self.name = name
        self.location = location
        self.available = True

class Restaurant:
    def __init__(self, name, location, menu):
        self.name = name
        self.location = location
        self.menu = menu

    def accepts_order(self):
        return random.random() < 0.8

class Order:
    STATUSES = ["INITIATED", "ACCEPTED", "DEPLOYED", "DELIVERED"]

    def __init__(self, user, restaurant, item):
        self.id = str(uuid4())[:8]
        self.user = user
        self.restaurant = restaurant
        self.item = item
        self.status = "INITIATED"
        self.driver = None
        self.priority = user.is_priority
        self.fee = 0

    def __lt__(self, other):
        return self.priority > other.priority  

    def update_status(self, new_status):
        self.status = new_status
        print(f"[Order {self.id}] Status updated to: {self.status}")

class DeliveryPlatform:
    def __init__(self):
        self.restaurants = []
        self.drivers = []
        self.order_queue = []

    def add_restaurant(self, restaurant):
        self.restaurants.append(restaurant)

    def add_driver(self, driver):
        self.drivers.append(driver)

    def distance(self, loc1, loc2):
        return ((loc1[0]-loc2[0])**2 + (loc1[1]-loc2[1])**2) ** 0.5

    def place_order(self, user, restaurant, item):
        if item not in restaurant.menu:
            print(f"{item} is not on the menu at {restaurant.name}")
            return None

        order = Order(user, restaurant, item)
        distance = self.distance(user.location, restaurant.location)
        order.fee = distance * (20 if user.is_priority else 10)
        print(f"[Order {order.id}] {item} ordered by {user.name} (Priority: {user.is_priority})")
        print(f"[Order {order.id}] Delivery fee: ₹{order.fee:.2f}")

        if restaurant.accepts_order():
            print(f"[Order {order.id}] Accepted by {restaurant.name}")
            heapq.heappush(self.order_queue, order)
            self.assign_driver_to_orders()
        else:
            print(f"[Order {order.id}] Rejected by {restaurant.name}")

        return order

    def find_nearest_available_driver(self, location):
        available_drivers = [d for d in self.drivers if d.available]
        if not available_drivers:
            return None
        return min(available_drivers, key=lambda d: self.distance(d.location, location))

    def assign_driver_to_orders(self):
        unassigned_orders = []
        while self.order_queue:
            order = heapq.heappop(self.order_queue)
            nearest_driver = self.find_nearest_available_driver(order.restaurant.location)
            if nearest_driver:
                nearest_driver.available = False
                order.driver = nearest_driver
                print(f"[Order {order.id}] Driver {nearest_driver.name} assigned.")
                threading.Thread(target=self.process_order, args=(order,)).start()
            else:
                print(f"[Order {order.id}] No available drivers. Waiting...")
                unassigned_orders.append(order)

        for order in unassigned_orders:
            heapq.heappush(self.order_queue, order)

    def process_order(self, order):
        order.update_status("ACCEPTED")
        time.sleep(1)
        order.update_status("DEPLOYED")
        time.sleep(1)
        order.update_status("DELIVERED")

        order.driver.available = True
        order.driver.location = order.user.location 
        print(f"[Order {order.id}] Completed by Driver {order.driver.name}\n")

        self.assign_driver_to_orders()

if __name__ == "__main__":
    platform = DeliveryPlatform()

    r1 = Restaurant("Pizza Palace", (0, 0), ["Margherita", "Pepperoni"])
    r2 = Restaurant("Burger Barn", (5, 5), ["Cheeseburger", "Veggie Burger"])
    platform.add_restaurant(r1)
    platform.add_restaurant(r2)

    platform.add_driver(Driver("Alice", (1, 1)))
    platform.add_driver(Driver("Bob", (6, 5)))
    platform.add_driver(Driver("Charlie", (3, 3)))

    u1 = User("Megha", (2, 2), is_priority=True)
    u2 = User("Rahul", (4, 4), is_priority=False)
    u3 = User("Priya", (1, 1), is_priority=True)
    u4 = User("Ankit", (6, 6), is_priority=False)

    platform.place_order(u1, r1, "Margherita")
    platform.place_order(u2, r2, "Cheeseburger")
    platform.place_order(u3, r2, "Veggie Burger")
    platform.place_order(u4, r1, "Pepperoni")


[Order 942e0173] Margherita ordered by Megha (Priority: True)
[Order 942e0173] Delivery fee: ₹56.57
[Order 942e0173] Accepted by Pizza Palace
[Order 942e0173] Driver Alice assigned.
[Order 942e0173] Status updated to: ACCEPTED
[Order 9196b242] Cheeseburger ordered by Rahul (Priority: False)
[Order 9196b242] Delivery fee: ₹14.14
[Order 9196b242] Accepted by Burger Barn
[Order 9196b242] Driver Bob assigned.
[Order 9196b242] Status updated to: ACCEPTED
[Order cbabd46e] Veggie Burger ordered by Priya (Priority: True)
[Order cbabd46e] Delivery fee: ₹113.14
[Order cbabd46e] Accepted by Burger Barn
[Order cbabd46e] Driver Charlie assigned.
[Order cbabd46e] Status updated to: ACCEPTED
[Order 5c29e741] Pepperoni ordered by Ankit (Priority: False)
[Order 5c29e741] Delivery fee: ₹84.85
[Order 5c29e741] Accepted by Pizza Palace
[Order 5c29e741] No available drivers. Waiting...
[Order 942e0173] Status updated to: DEPLOYED[Order 9196b242] Status updated to: DEPLOYED
[Order cbabd46e] Status updated t